# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
compras = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})

compras

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot_table = pd.pivot_table(compras, columns = 'CustomerID',index = 'ProductName', fill_value = 0, values = 'Quantity' )
pivot_table.head()

pivot_table = pd.DataFrame(pivot_table,index = list(pivot_table.index),columns=list(pivot_table.columns))
pivot_table

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0
Yogurt - French Vanilla,1,0,0,1,0,0,2,0,0,1,...,0,0,25,0,0,0,0,0,0,25


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
pivot_table_t = pivot_table.T
pivot_table_t

,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0
98185,0,0,25,25,0,25,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0


In [7]:
distances = pd.DataFrame(1 / (1 + squareform(pdist(pivot_table_t, metric = 'euclidean'))),index=pivot_table_t.index, columns=pivot_table_t.index)
distances

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266
98185,0.004167,0.004166,0.004179,0.004175,0.004177,0.004167,0.004168,0.004176,0.004172,0.004161,...,0.003311,0.003162,0.003255,0.003406,0.003172,0.003255,0.003345,0.003277,1.000000,0.003288


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
# seleccionaremos los mas cercanos al numero 669
similarities = distances[669].sort_values(ascending = False)[1:]
top5_similares = similarities.nlargest(5)
id_similares = list(similarities.nlargest(5).index)
print(top5_similares)
print('')
print(id_similares)

id_similares

3535    0.084959
3317    0.083651
639     0.083020
1008    0.082100
3305    0.081800
Name: 669, dtype: float64

[3535, 3317, 639, 1008, 3305]


[3535, 3317, 639, 1008, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
similar_costmer = pivot_table_t.filter(id_similares,axis=0)
similar_costmer = similar_costmer.T
similar_costmer

,3535,3317,639,1008,3305
Anchovy Paste - 56 G Tube,1,0,1,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0
Appetizer - Mushroom Tart,0,0,1,0,0
Appetizer - Sausage Rolls,0,0,0,0,0
Apricots - Dried,1,0,0,0,0
...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0
Yoghurt Tubes,1,0,0,1,0
"Yogurt - Blueberry, 175 Gr",0,0,0,0,0
Yogurt - French Vanilla,0,0,0,0,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
for name, score in dict(top5_similares).items():
    similar_costmer[name]=similar_costmer[name]*score

similar_costmer['Total']=similar_costmer.sum(axis=1)
similar_costmer.sort_values('Total',ascending=False)

,3535,3317,639,1008,3305,Total
"Veal - Inside, Choice",0.000000,0.083651,0.08302,0.0821,0.0818,0.330570
"Pepper - Black, Whole",0.084959,0.167302,0.00000,0.0000,0.0000,0.252261
Cheese - Cambozola,0.084959,0.000000,0.08302,0.0821,0.0000,0.250078
Wine - Blue Nun Qualitatswein,0.084959,0.000000,0.00000,0.0821,0.0818,0.248859
"Sauce - Gravy, Au Jus, Mix",0.000000,0.083651,0.08302,0.0821,0.0000,0.248770
...,...,...,...,...,...,...
Chicken - Wieners,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000
"Chicken - Leg, Boneless",0.000000,0.000000,0.00000,0.0000,0.0000,0.000000
Chef Hat 20cm,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000
"Pepsi - Diet, 355 Ml",0.000000,0.000000,0.00000,0.0000,0.0000,0.000000


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
costumer_669 = pivot_table_t[pivot_table_t.index == 669].T
costumer_669

,669
Anchovy Paste - 56 G Tube,0
"Appetizer - Mini Egg Roll, Shrimp",0
Appetizer - Mushroom Tart,0
Appetizer - Sausage Rolls,0
Apricots - Dried,0
...,...
Yeast Dry - Fermipan,0
Yoghurt Tubes,0
"Yogurt - Blueberry, 175 Gr",0
Yogurt - French Vanilla,1


In [12]:
df = pd.concat([costumer_669,similar_costmer],axis=1)
recomendaciones = df[df[669]==0]
recomendaciones['Total'].nlargest(5)

Veal - Inside, Choice            0.330570
Pepper - Black, Whole            0.252261
Wine - Blue Nun Qualitatswein    0.248859
Wine - Ej Gallo Sierra Valley    0.247250
Sardines                         0.246619
Name: Total, dtype: float64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
ID_users = list(distances.columns)

def user_recomendation(user_ID):
    # Obtencion de las distancias
    distances = pd.DataFrame(1 / (1 + squareform(pdist(pivot_table_t, metric = 'euclidean'))),index=pivot_table_t.index, columns=pivot_table_t.index)
    
    # Obtenemos los similares
    similarities = distances[user_ID].sort_values(ascending = False)[1:]
    top5_similares = similarities.nlargest(5)
    id_similares = list(similarities.nlargest(5).index)

    # Productos comprados de los usuarios similares
    similar_costmer = pivot_table_t.filter(id_similares,axis=0)
    similar_costmer = similar_costmer.T
    similar_costmer

    # Generamos las calificacion por la distancia entre usuarios
    for name, score in dict(top5_similares).items():
        similar_costmer[name]=similar_costmer[name]*score

    similar_costmer['Total']=similar_costmer.sum(axis=1)
    similar_costmer.sort_values('Total',ascending=False)

    # Creamos una tabla solo del usuario que estamos revisando
    costumer = pivot_table_t[pivot_table_t.index == user_ID].T
    costumer

    # generamos un df con las recomendaciones por usuario
    df = pd.concat([costumer,similar_costmer],axis=1)
    recomendaciones = df[df[user_ID]==0]
    
    return recomendaciones['Total'].nlargest(5)

recomendaciones = {i:user_recomendation(i) for i in ID_users }

recomendaciones

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
recomendaciones_df = pd.DataFrame(recomendaciones).fillna(0)
recomendaciones_df = recomendaciones_df.T
recomendaciones_df

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
def user_recomendation(user_ID):
    # Obtencion de las distancias
    distances = pd.DataFrame(1 / (1 + squareform(pdist(pivot_table_t, metric = 'cityblock'))),index=pivot_table_t.index, columns=pivot_table_t.index)
    
    # Obtenemos los similares
    similarities = distances[user_ID].sort_values(ascending = False)[1:]
    top5_similares = similarities.nlargest(5)
    id_similares = list(similarities.nlargest(5).index)

    # Productos comprados de los usuarios similares
    similar_costmer = pivot_table_t.filter(id_similares,axis=0)
    similar_costmer = similar_costmer.T
    similar_costmer

    # Generamos las calificacion por la distancia entre usuarios
    for name, score in dict(top5_similares).items():
        similar_costmer[name]=similar_costmer[name]*score

    similar_costmer['Total']=similar_costmer.sum(axis=1)
    similar_costmer.sort_values('Total',ascending=False)

    # Creamos una tabla solo del usuario que estamos revisando
    costumer = pivot_table_t[pivot_table_t.index == user_ID].T
    costumer

    # generamos un df con las recomendaciones por usuario
    df = pd.concat([costumer,similar_costmer],axis=1)
    recomendaciones = df[df[user_ID]==0]
    
    return recomendaciones['Total'].nlargest(5)

recomendaciones = {i:user_recomendation(i) for i in ID_users }

recomendaciones